# Salesforce ETL Project - Part 3 (Staff & Staff Assignment)

In [1]:
#!pip install simple_salesforce
#!pip install pymysql
#!pip install sqlalchemy

In [2]:
import json
from sqlalchemy import create_engine
import datetime as dt
from datetime import date
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [3]:
# Make sure to use your own `config.py` file. Consider ensuring that these variable names are in sync
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [4]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [5]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

## Prepare ETL for the Staff data

#### Staff table on MySQL
![Salesforce ETL Project - MySQL Table Staff](Images/MySQL_Table_Staff.jpg)

In [6]:
staff_data_df = pd.read_sql("SELECT * FROM staff", conn)
staff_data_df.head()

,ID_Staff,EmployeeID,LastName,FirstName,MiddleName,BirthDate
0,1,000184220,Luongo,Darick,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Rodney,Heather,Nicole,1994-06-07


In [7]:
# staff_data_df
staff_data_df['BirthDate'] = pd.to_datetime(staff_data_df['BirthDate']).dt.date
staff_data_df.head()

,ID_Staff,EmployeeID,LastName,FirstName,MiddleName,BirthDate
0,1,000184220,Luongo,Darick,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Rodney,Heather,Nicole,1994-06-07


In [8]:
staff_data_df.rename(columns={
    'EmployeeID':'Employee_ID__c',
    'FirstName':'First_Name__c',
    'LastName':'Last_Name__c',
    'MiddleName':'Middle_Name__c',
    'BirthDate':'Birth_Date__c'    
}, inplace=True)
staff_data_df.head()

,ID_Staff,Employee_ID__c,Last_Name__c,First_Name__c,Middle_Name__c,Birth_Date__c
0,1,000184220,Luongo,Darick,Nico,1995-07-05
1,2,000130109,Sanford,Gemini,Blair,1992-04-22
2,3,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,4,000159108,Rodney,Heather,Nicole,1994-06-07


In [9]:
staff_data_df = staff_data_df[['Employee_ID__c', 'Last_Name__c', 'First_Name__c', 'Middle_Name__c', 'Birth_Date__c']]
staff_data_df.head()

,Employee_ID__c,Last_Name__c,First_Name__c,Middle_Name__c,Birth_Date__c
0,000184220,Luongo,Darick,Nico,1995-07-05
1,000130109,Sanford,Gemini,Blair,1992-04-22
2,000160655,Williams,Dartanion,De Angelo,1993-05-21
3,000159108,Rodney,Heather,Nicole,1994-06-07


In [10]:
staff_data_records = staff_data_df.to_dict('records')
staff_data_records

[{'Employee_ID__c': '000184220',
  'Last_Name__c': 'Luongo',
  'First_Name__c': 'Darick',
  'Middle_Name__c': 'Nico',
  'Birth_Date__c': datetime.date(1995, 7, 5)},
 {'Employee_ID__c': '000130109',
  'Last_Name__c': 'Sanford',
  'First_Name__c': 'Gemini',
  'Middle_Name__c': 'Blair',
  'Birth_Date__c': datetime.date(1992, 4, 22)},
 {'Employee_ID__c': '000160655',
  'Last_Name__c': 'Williams',
  'First_Name__c': 'Dartanion',
  'Middle_Name__c': 'De Angelo',
  'Birth_Date__c': datetime.date(1993, 5, 21)},
 {'Employee_ID__c': '000159108',
  'Last_Name__c': 'Rodney',
  'First_Name__c': 'Heather',
  'Middle_Name__c': 'Nicole',
  'Birth_Date__c': datetime.date(1994, 6, 7)}]

## Insert `Staff` Records into Salesforce

In [11]:
for rec in staff_data_records:
   
    record = {
        'Employee_ID__c': rec['Employee_ID__c'],
        'Last_Name__c': rec['Last_Name__c'],
        'First_Name__c': rec['First_Name__c'],
        'Middle_Name__c': rec['Middle_Name__c'],
        'Birth_Date__c': str(rec['Birth_Date__c'])
    }
    
    try:
        sf.Staff__c.create(record)
    except Exception as e:
        print(e)

In [12]:
# Bulk 
#sf.bulk.Staff__c.insert(staff_data_records)

#### Staff object / table on Salesforce
![Salesforce ETL Project - Salesforce Object Staff](Images/SF_Object_Staff.jpg)

## Create Staff Lookup Table
You will use this later to crosswalk the course code with the primary key from the `Course` table

It is important to note that we will be querying **Salesforce** to retrieve the record IDs 

In [13]:
staff_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT Employee_ID__c, Name FROM Staff__c")
for row in data:
    rec = {
        'ID_Employee__c': row['Name'], # this is a critical line of code
        'Employee_ID__c': row['Employee_ID__c']
    }
    staff_lookup_list.append(rec)
    
staff_lookup_list

[{'ID_Employee__c': 'a034x000002jl5i', 'Employee_ID__c': '000130109'},
 {'ID_Employee__c': 'a034x000002jl9U', 'Employee_ID__c': '000159108'},
 {'ID_Employee__c': 'a034x000002jl9K', 'Employee_ID__c': '000184220'},
 {'ID_Employee__c': 'a034x000002jl9P', 'Employee_ID__c': '000160655'}]

In [14]:
staff_lookup_df = pd.DataFrame(staff_lookup_list)
staff_lookup_df

,ID_Employee__c,Employee_ID__c
0,a034x000002jl5i,000130109
1,a034x000002jl9U,000159108
2,a034x000002jl9K,000184220
3,a034x000002jl9P,000160655


## Create Class & Course Joint Lookup Table

It is important to note that we will be querying **Salesforce** to retrieve the record IDs 

In [15]:
course_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT Course_Code__c, Name FROM Course__c")
for row in data:
    rec = {
        'ID_Course__c': row['Name'], # this is a critical line of code
        'Course_Code__c': row['Course_Code__c'] # this is not necessary even though it's also a unique identifier
    }
    course_lookup_list.append(rec)
    
course_lookup_list

[{'ID_Course__c': 'a004x000003VTav', 'Course_Code__c': 'BC-UIUX'},
 {'ID_Course__c': 'a004x000003VTbF', 'Course_Code__c': 'CIS-430'},
 {'ID_Course__c': 'a004x000003VTb5', 'Course_Code__c': 'CIS-405'},
 {'ID_Course__c': 'a004x000003VTbA', 'Course_Code__c': 'CIS-438'},
 {'ID_Course__c': 'a004x000003VTal', 'Course_Code__c': 'BC-DATAVIZ'},
 {'ID_Course__c': 'a004x000003VTb0', 'Course_Code__c': 'CIS-349'},
 {'ID_Course__c': 'a004x000003VTbK', 'Course_Code__c': 'CIS-435'},
 {'ID_Course__c': 'a004x000003VTaq', 'Course_Code__c': 'BC-WEBDEV'}]

In [16]:
course_lookup_df = pd.DataFrame(course_lookup_list)
course_lookup_df

,ID_Course__c,Course_Code__c
0,a004x000003VTav,BC-UIUX
1,a004x000003VTbF,CIS-430
2,a004x000003VTb5,CIS-405
3,a004x000003VTbA,CIS-438
4,a004x000003VTal,BC-DATAVIZ
5,a004x000003VTb0,CIS-349
6,a004x000003VTbK,CIS-435
7,a004x000003VTaq,BC-WEBDEV


In [17]:
class_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
# The Salesforce query language is called SOQL 
data = sf.query_all_iter("SELECT Name, ID_Course__c, Section__c FROM Class__c")

for row in data:  # NEED TO UNDERSTAND WHY DART USED RECORDS HERE; PREVIOUSLY, WE HAD NOT
    rec = {
        'ID_Class__c': row['Name'], # this is a critical line of code
        'ID_Course__c': row['ID_Course__c'],
        'Section__c': row['Section__c']
    }
    class_lookup_list.append(rec)
    
# Temporary workaround to strip last 3 characters from ID_Course__c
for rec in class_lookup_list:
    rec['ID_Course__c'] = rec['ID_Course__c'][:-3]
    
class_lookup_list

[{'ID_Class__c': 'a014x000008WraB',
  'ID_Course__c': 'a004x000003VTal',
  'Section__c': 'GWDC201805DATA3'},
 {'ID_Class__c': 'a014x000008Wra6',
  'ID_Course__c': 'a004x000003VTal',
  'Section__c': 'GWU-ARL-DATA-PT-09-0'},
 {'ID_Class__c': 'a014x000008WraL',
  'ID_Course__c': 'a004x000003VTav',
  'Section__c': 'GWARL201905UIUX3'},
 {'ID_Class__c': 'a014x000008WraG',
  'ID_Course__c': 'a004x000003VTaq',
  'Section__c': 'GWARL201905WEB3'}]

In [18]:
class_lookup_df = pd.DataFrame(class_lookup_list)
class_lookup_df

,ID_Class__c,ID_Course__c,Section__c
0,a014x000008WraB,a004x000003VTal,GWDC201805DATA3
1,a014x000008Wra6,a004x000003VTal,GWU-ARL-DATA-PT-09-0
2,a014x000008WraL,a004x000003VTav,GWARL201905UIUX3
3,a014x000008WraG,a004x000003VTaq,GWARL201905WEB3


In [19]:
class_course_joint_lookup_df = pd.merge(course_lookup_df, class_lookup_df, on=['ID_Course__c'])
class_course_joint_lookup_df

,ID_Course__c,Course_Code__c,ID_Class__c,Section__c
0,a004x000003VTav,BC-UIUX,a014x000008WraL,GWARL201905UIUX3
1,a004x000003VTal,BC-DATAVIZ,a014x000008WraB,GWDC201805DATA3
2,a004x000003VTal,BC-DATAVIZ,a014x000008Wra6,GWU-ARL-DATA-PT-09-0
3,a004x000003VTaq,BC-WEBDEV,a014x000008WraG,GWARL201905WEB3


## Create Staff Assignment DataFrame

#### Staff Assignment table on MySQL
![Salesforce ETL Project - MySQL Table Staff Assignment](Images/MySQL_Table_Staff_Assignment.jpg)

In [20]:
# Query the `Class Participant` table from MySQL
query = '''
    SELECT 
        st.EmployeeID,        
        cl.ID_Course,
        co.CourseCode,
        cl.Section,
        sa.*
    FROM 
        staffassignment sa
        INNER JOIN staff st
        ON sa.ID_Staff = st.ID_Staff
        INNER JOIN class cl
        ON sa.ID_Class = cl.ID_Class
        INNER JOIN course co
        ON cl.ID_Course = co.ID_Course

'''

staff_assignment_data_df = pd.read_sql(query, conn)
staff_assignment_data_df.head()

,EmployeeID,ID_Course,CourseCode,Section,ID_StaffAssignment,ID_Staff,ID_Class,Role,StartDate,EndDate
0,000184220,1,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,1,1,1,Teacher Assistant,2020-09-16,None
1,000159108,1,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,2,4,1,Teacher Assistant,2020-09-16,None
2,000160655,1,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,3,3,1,Instructor,2020-09-16,None


In [21]:
staff_assignment_data_df.drop(['ID_Course', 'ID_StaffAssignment', 'ID_Staff', 'ID_Class'], axis=1, inplace=True)

staff_assignment_data_df.head()

,EmployeeID,CourseCode,Section,Role,StartDate,EndDate
0,000184220,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,Teacher Assistant,2020-09-16,None
1,000159108,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,Teacher Assistant,2020-09-16,None
2,000160655,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,Instructor,2020-09-16,None


In [22]:
staff_assignment_data_df.rename(columns={
    'EmployeeID':'Employee_ID__c',    
    'Section': 'Section__c',
    'CourseCode': 'Course_Code__c',
    'Role': 'Role__c',
    'StartDate':'Start_Date__c',
    'EndDate':'End_Date__c'
}, inplace=True)

staff_assignment_data_df.head()

,Employee_ID__c,Course_Code__c,Section__c,Role__c,Start_Date__c,End_Date__c
0,000184220,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,Teacher Assistant,2020-09-16,None
1,000159108,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,Teacher Assistant,2020-09-16,None
2,000160655,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,Instructor,2020-09-16,None


## Join the Staff Assignment DataFrame with the Course & Class Joint Lookup Table
This join is necessary to successfully lookup the foreign key for the Class table 

In [23]:
staff_assignment_data_df1 = pd.merge(staff_assignment_data_df, class_course_joint_lookup_df, on=['Section__c', 'Course_Code__c'])

staff_assignment_data_df1.head()

,Employee_ID__c,Course_Code__c,Section__c,Role__c,Start_Date__c,End_Date__c,ID_Course__c,ID_Class__c
0,000184220,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,Teacher Assistant,2020-09-16,None,a004x000003VTal,a014x000008Wra6
1,000159108,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,Teacher Assistant,2020-09-16,None,a004x000003VTal,a014x000008Wra6
2,000160655,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,Instructor,2020-09-16,None,a004x000003VTal,a014x000008Wra6


## Join the Staff Assignment DataFrame with the Staff lookup table
This join is necessary to successfully lookup the foreign key for the Course table 

In [24]:
staff_assignment_data_df2 = pd.merge(staff_assignment_data_df1, staff_lookup_df, on=['Employee_ID__c'])

staff_assignment_data_df2.head()

,Employee_ID__c,Course_Code__c,Section__c,Role__c,Start_Date__c,End_Date__c,ID_Course__c,ID_Class__c,ID_Employee__c
0,000184220,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,Teacher Assistant,2020-09-16,None,a004x000003VTal,a014x000008Wra6,a034x000002jl9K
1,000159108,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,Teacher Assistant,2020-09-16,None,a004x000003VTal,a014x000008Wra6,a034x000002jl9U
2,000160655,BC-DATAVIZ,GWU-ARL-DATA-PT-09-0,Instructor,2020-09-16,None,a004x000003VTal,a014x000008Wra6,a034x000002jl9P


In [25]:
staff_assignment_data_df = staff_assignment_data_df2
staff_assignment_data_df.drop(['Employee_ID__c', 'Course_Code__c', 'Section__c', 'ID_Course__c'], axis=1, inplace=True)

staff_assignment_data_df.head()

,Role__c,Start_Date__c,End_Date__c,ID_Class__c,ID_Employee__c
0,Teacher Assistant,2020-09-16,None,a014x000008Wra6,a034x000002jl9K
1,Teacher Assistant,2020-09-16,None,a014x000008Wra6,a034x000002jl9U
2,Instructor,2020-09-16,None,a014x000008Wra6,a034x000002jl9P


In [26]:
staff_assignment_data_records = staff_assignment_data_df.to_dict(orient='records')
staff_assignment_data_records

[{'Role__c': 'Teacher Assistant',
  'Start_Date__c': datetime.date(2020, 9, 16),
  'End_Date__c': None,
  'ID_Class__c': 'a014x000008Wra6',
  'ID_Employee__c': 'a034x000002jl9K'},
 {'Role__c': 'Teacher Assistant',
  'Start_Date__c': datetime.date(2020, 9, 16),
  'End_Date__c': None,
  'ID_Class__c': 'a014x000008Wra6',
  'ID_Employee__c': 'a034x000002jl9U'},
 {'Role__c': 'Instructor',
  'Start_Date__c': datetime.date(2020, 9, 16),
  'End_Date__c': None,
  'ID_Class__c': 'a014x000008Wra6',
  'ID_Employee__c': 'a034x000002jl9P'}]

## Insert `Staff Assignment` Records into Salesforce

In [28]:
for rec in staff_assignment_data_records:
 
    record = {
        'ID_Class__c': rec['ID_Class__c'],
        'ID_Employee__c': rec['ID_Employee__c'],
        'Role__c': rec['Role__c'],
        'Start_Date__c': str(rec['Start_Date__c']),
        'End_Date__c': rec['End_Date__c'],
    }
    
    try:
        sf.Staff_Assignment__C.create(record)
    except Exception as e:
        print(e)

In [ ]:
# Bulk 
#sf.bulk.Staff_Assignment__c.insert(staff_assignment_data_records)

#### Staff Assignment object / table on Salesforce
![Salesforce ETL Project - Salesforce Object Staff Assignment](Images/SF_Object_Staff_Assignment.jpg)

## Example of Deleting Records

Select the IDs of the records first and then process the results.

Ultimately, you want a list of IDs in the end.


#### Delete Staff object/table records in Salesforce databootcamp/gwsis

In [ ]:
staff_records = sf.query("SELECT Id FROM Staff__c")
recs_to_delete = [{'Id': r['Id']} for r in staff_records['records']]
recs_to_delete

In [ ]:
#sf.bulk.Course__c.delete(recs_to_delete)

In [ ]:
for rec in recs_to_delete:
    try:
        sf.Staff__c.delete(rec['Id'])
    except Exception as e:
        print(e)

#### Delete Staff Assignment object/table records in Salesforce databootcamp/gwsis

In [ ]:
staff_assignment_records = sf.query("SELECT Id FROM Staff_Assignment__c")
recs_to_delete = [{'Id': r['Id']} for r in staff_assignment_records['records']]
recs_to_delete

In [ ]:
#sf.bulk.Course__c.delete(recs_to_delete)

In [ ]:
for rec in recs_to_delete:
    try:
        sf.Staff_Assignment__c.delete(rec['Id'])
    except Exception as e:
        print(e)